# Create Vector store instance for K collection

In [21]:
DB_NAME = "project_2_v2"
K_V_POISON_COLLECTION_NAME = "K_V_poison_dataset_0_6"
K_V_POISON_SYSTEM_COLLECTION_NAME_0_6 = "K_V_poison_system_0_6"
QUERY_COLLECTION = "query_collection"
TOGETHER_MODEL_NAME = "togethercomputer/m2-bert-80M-8k-retrieval"
URI = "http://localhost:19530"

NUMBER_OF_SAMPLE = 100

FILE_PATH = 'data/filtered_ReposVul_samples.jsonl'

In [22]:
from langchain_milvus import BM25BuiltInFunction, Milvus
from langchain_openai import OpenAIEmbeddings
import getpass
import os

In [23]:
import os
from dotenv import load_dotenv
load_dotenv()
# Lấy JINA_KEY từ biến môi trường hoặc cung cấp giá trị mặc định
os.environ["JINA_KEY"] = os.getenv("JINA_KEY")
os.environ["TOGETHER_API_KEY"] = os.getenv("TOGETHER_API_KEY")


In [24]:
from langchain_community.embeddings.jina import JinaEmbeddings
from langchain_together import TogetherEmbeddings
# Khởi tạo JinaEmbedding với API key và mô hình jina-embeddings-v3
# jina_embeddings = JinaEmbeddings(
#     jina_api_key=os.environ["JINA_KEY"], 
#     model_name="jina-embeddings-v3"
# )

together_embeddings = TogetherEmbeddings(
    model=TOGETHER_MODEL_NAME,
    together_api_key=os.environ["TOGETHER_API_KEY"]
)

# Khởi tạo Milvus vectorstore
K_V_system_vectorstore_0_6 = Milvus(
    embedding_function=together_embeddings,
    connection_args={"uri": URI, "token": "root:Milvus", "db_name": DB_NAME},
    index_params={"index_type": "FLAT", "metric_type": "L2"},
    consistency_level="Strong",
    drop_old=False,  # set to True if seeking to drop the collection with that name if it exists
    auto_id=True,
    collection_name=K_V_POISON_SYSTEM_COLLECTION_NAME_0_6,
)

query_collection = Milvus(
    embedding_function=together_embeddings,
    connection_args={"uri": URI, "token": "root:Milvus", "db_name": DB_NAME},
    index_params={"index_type": "FLAT", "metric_type": "L2"},
    consistency_level="Strong",
    drop_old=False,  # set to True if seeking to drop the collection with that name if it exists
    auto_id=True,
    collection_name=QUERY_COLLECTION,
)


# Get query vector from query collection

In [29]:
# K_EXPECTED_METADATA_FIELDS = ['index','vector' ,'cve_id', 'cwe_id', 'cve_language']

pk_field_name = "pk"

# K_fields_to_retrieve = [K_pk_field_name] + K_EXPECTED_METADATA_FIELDS

query_expression = f"{pk_field_name} >= 0"

query_limit = 10000


all_query_data = []
try:
    # Truy cập đối tượng collection pymilvus bên trong vectorstore
    # Lưu ý: việc truy cập thuộc tính .col có thể thay đổi trong các phiên bản LangChain tương lai
    if hasattr(query_collection, 'col') and query_collection.col is not None:
        milvus_collection = query_collection.col
        # Thực hiện query trực tiếp trên đối tượng collection
        results = milvus_collection.query(
            expr=query_expression,
            # output_fields=K_fields_to_retrieve,
            output_fields=["*"],
            limit=query_limit,
            consistency_level="Strong" # Đảm bảo đọc dữ liệu mới nhất
        )
        all_query_data = results
    else:
        print("Lỗi: Không thể truy cập đối tượng collection pymilvus (.col) bên trong vectorstore.")

except Exception as e:
    print(f"\nLỗi khi thực hiện query trên Milvus: {e}")

# 4. In ra kết quả
if all_query_data:
    for i, sample_data in enumerate(all_query_data[:5]):
        print(f"\nSample {i + 1}:")
        # sample_data ở đây là một dictionary chứa các trường bạn yêu cầu lấy ra
        print(sample_data)
else:
    print("\nQuery không trả về kết quả nào hoặc đã xảy ra lỗi.")


Sample 1:
{'text': 'Disable some of the FastCopy fast paths since they are racy.', 'pk': 457611612880310199, 'vector': [-0.03535331, 0.29097983, 0.13401379, 0.30674756, -0.098402254, -0.060393035, -0.018799804, -0.061040744, -0.012064579, -0.07780195, -0.09368918, 0.08078216, -0.35229802, -0.17163241, -0.0515353, -0.2176337, 0.24905568, -0.088495485, -0.15365829, -0.06900435, -0.17300035, -0.07667153, 0.021881253, 0.19152175, -0.2050516, -0.19392513, 0.20412743, -0.00831759, -0.07405321, 0.003882255, -0.12889928, -0.15873846, -0.25240248, -0.18981336, -0.16002198, 0.022189334, 0.16193107, 0.2019678, -0.10369671, 0.1841409, 0.092184074, -0.050505884, 0.10303757, -0.307132, -0.10235461, -0.05883311, -0.03263402, -0.19117926, -0.040035628, 0.26483566, 0.14853641, 0.0078119235, -0.10527276, -0.14247704, -0.07846062, 0.07190965, 0.25482118, 0.38914138, -0.17758502, 0.12624002, 0.090786316, 0.18119341, -0.006138153, 0.30495447, -0.15270142, -0.17234503, -0.07625523, -0.1875528, -0.09088564,

In [32]:
list_of_query_vector = []
for query in all_query_data:
    list_of_query_vector.append(query["vector"])

# Retrie code in system by query vector

In [58]:
search_params = {
    "metric_type": "L2",  # Hoặc "IP"
    "params": {},         # Ví dụ: {"nprobe": 10} nếu dùng IVF_FLAT
}

search_results_pymilvus = []

try:
    # *** THỰC HIỆN TÌM KIẾM BẰNG PYMILVUS.SEARCH ***
    search_results_pymilvus = K_V_system_vectorstore_0_6.col.search(
        data=list_of_query_vector,
        anns_field='vector',
        param=search_params,
        limit=1,                  # Chỉ lấy 1 kết quả
        output_fields=["*"] # <--- Yêu cầu trả về các trường này
    )

except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()

In [59]:
print(search_results_pymilvus[0][0].data_source)

K


In [63]:
count_v = 0
for res in search_results_pymilvus:
    if res[0].data_source == 'V':
        count_v += 1
        
print(f"Number of V in the result: {count_v}")

Number of V in the result: 7


# Caculate Vulnerability Rate in Retrieved Code (VRRC):